## AWS tutorial 'Capturing Table Activity with DynamoDB Streams'

When a record is inserted to a dynamoDB table, the lambda publisha message. An email is subscribed to the topic.

Setting up:
- create table
- create role, policy
- create topic, subscribe topic
- create lambda
- create lambda trig

<a href='#test_rec'>Test: adding records to DynamoDB</a>

https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Streams.Lambda.Tutorial.html?shortFooter=true#Streams.Lambda.Tutorial.CreateTable

<b>curl MQTT</b>
<code>
    # sample
    curl --tlsv1.2 --cacert root-CA.crt --cert 4b7828d2e5-certificate.pem.crt --key 4b7828d2e5-private.pem.key -X POST -d "{ \"message\": \"Hello, world\" }" "https://a1pn10j0v8htvw.iot.us-east-1.amazonaws.com:8443/topics/my/topic"
    # IoTTestDevice
    curl --tlsv1.2 --cacert root-CA.crt --cert IoTTestDevice.cert.pem --key IoTTestDevice.private.key -X POST -d "{ \"message\": \"Hello, world\" }" "https://a3a13fa4bs6nw5-ats.iot.us-west-2.amazonaws.com:8443/topics/my/topic"
    
</code>

In [5]:
aws dynamodb create-table --table-name DeviceInfoTable \
    --attribute-definitions AttributeName=DeviceId,AttributeType=S AttributeName=Timestamp,AttributeType=S \
    --key-schema AttributeName=DeviceId,KeyType=HASH  AttributeName=Timestamp,KeyType=RANGE \
    --provisioned-throughput ReadCapacityUnits=1,WriteCapacityUnits=1 \
    --stream-specification StreamEnabled=true,StreamViewType=NEW_AND_OLD_IMAGES

{
    "TableDescription": {
        "AttributeDefinitions": [
            {
                "AttributeName": "DeviceId",
                "AttributeType": "S"
            },
            {
                "AttributeName": "Timestamp",
                "AttributeType": "S"
            }
        ],
        "TableName": "DeviceInfoTable",
        "KeySchema": [
            {
                "AttributeName": "DeviceId",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "Timestamp",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "CREATING",
        "CreationDateTime": 1572371391.744,
        "ProvisionedThroughput": {
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 1,
            "WriteCapacityUnits": 1
        },
        "TableSizeBytes": 0,
        "ItemCount": 0,
        "TableArn": "arn:aws:dynamodb:us-west-2:408575476948:table/DeviceInfoTable",
        "TableId": "8cdc42dc-a5d4-4e6a

In [6]:
# change wcu and rcu
# aws dynamodb update-table --table-name BarkTable --provisioned-throughput ReadCapacityUnits=1,WriteCapacityUnits=1

#describe
aws dynamodb describe-table --table-name DeviceInfoTable

{
    "Table": {
        "AttributeDefinitions": [
            {
                "AttributeName": "DeviceId",
                "AttributeType": "S"
            },
            {
                "AttributeName": "Timestamp",
                "AttributeType": "S"
            }
        ],
        "TableName": "DeviceInfoTable",
        "KeySchema": [
            {
                "AttributeName": "DeviceId",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "Timestamp",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "ACTIVE",
        "CreationDateTime": 1572371391.744,
        "ProvisionedThroughput": {
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 1,
            "WriteCapacityUnits": 1
        },
        "TableSizeBytes": 0,
        "ItemCount": 0,
        "TableArn": "arn:aws:dynamodb:us-west-2:408575476948:table/DeviceInfoTable",
        "TableId": "8cdc42dc-a5d4-4e6a-84fc-84ba84f

In [7]:
# create trust-relation role
aws iam create-role --role-name DeviceLambdaRole --path "/service-role/" \
    --assume-role-policy-document file://trust-relationship.json

{
    "Role": {
        "Path": "/service-role/",
        "RoleName": "DeviceLambdaRole",
        "RoleId": "AROAV6IH7ATKH5IG4CBXD",
        "Arn": "arn:aws:iam::408575476948:role/service-role/DeviceLambdaRole",
        "CreateDate": "2019-10-29T18:00:06Z",
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "lambda.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }
    }
}


In [12]:
# create policy and attach it to the role
# aws iam put-role-policy --role-name DeviceLambdaRole --policy-name DeviceLambdaRolePolicy \
#    --policy-document file://role-policy.json

# describe
aws iam get-role-policy --role-name DeviceLambdaRole --policy-name DeviceLambdaRolePolicy

{
    "RoleName": "DeviceLambdaRole",
    "PolicyName": "DeviceLambdaRolePolicy",
    "PolicyDocument": {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "lambda:InvokeFunction",
                "Resource": "arn:aws:lambda:us-west-2:408575476948:function:publishDeviceStatus*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents"
                ],
                "Resource": "arn:aws:logs:us-west-2:408575476948:*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "dynamodb:DescribeStream",
                    "dynamodb:GetRecords",
                    "dynamodb:GetShardIterator",
                    "dynamodb:ListStreams"
                ],
                "Resource": "arn:aws:dyna

In [13]:
# create a sns topic
aws sns create-topic --name deviceStatusTopic

{
    "TopicArn": "arn:aws:sns:us-west-2:408575476948:deviceStatusTopic"
}


In [14]:
# create topic subscription
aws sns subscribe --topic-arn arn:aws:sns:us-west-2:408575476948:deviceStatusTopic \
    --protocol email --notification-endpoint coderming@gmail.com

{
    "SubscriptionArn": "pending confirmation"
}


#### I create function 'publishDeviceStatus' at console

to use aws cli (not tested) : 
- edit publishDeviceStatus.py file
- then run:
<code>
$ zip publishDeviceStatus.zip publishDeviceStatus.py
$ aws lambda create-function \
    --region us-west-2 \
    --function-name publishDeviceStatus \
    --zip-file fileb://publishDeviceStatus.zip \
    --role arn:aws:iam::408575476948:role/service-role/DeviceLambdaRole \
    --handler publishDeviceStatus.handler \
    --timeout 5 \
    --runtime python3.7
</code>

In [15]:
# create dynamodb trig on the lambda
aws lambda create-event-source-mapping \
    --region us-west-2 \
    --function-name publishDeviceStatus \
    --event-source arn:aws:dynamodb:us-west-2:408575476948:table/DeviceInfoTable/stream/2019-10-29T17:49:51.744 \
    --batch-size 1 \
    --starting-position TRIM_HORIZON

{
    "UUID": "f3063ab1-9870-44ce-9130-7bb5058b04c7",
    "BatchSize": 1,
    "MaximumBatchingWindowInSeconds": 0,
    "EventSourceArn": "arn:aws:dynamodb:us-west-2:408575476948:table/DeviceInfoTable/stream/2019-10-29T17:49:51.744",
    "FunctionArn": "arn:aws:lambda:us-west-2:408575476948:function:publishDeviceStatus",
    "LastModified": 1572380863.114,
    "LastProcessingResult": "No records processed",
    "State": "Creating",
    "StateTransitionReason": "User action"
}


###  <a name='test_rec'>Test the flow by adding records to dynamodb, invalid json </a>

In [18]:

aws dynamodb put-item --table-name DeviceInfoTable \
 --item DeviceId={S="manually"},Timestamp={S="2019-10-29:13:10:36"},Status={S="OK"}
aws dynamodb put-item --table-name DeviceInfoTable \
 --item DeviceId={S="manually"},Timestamp={S="2019-10-29:13:10:46"},Status={S="Bad"}


### Manually Configured IoT connection to DynamoDB at console

- create a rule 'deviceStatus'
- 'Add action', select DynamoDB 'insert a message into a DynamoDB table':
  <code>
             Table name : DeviceInfoTable
          Partition key : DeviceId             // DynamoDB key
    Partition key value : ${myid} 
               Sort key : TimeStamp            // DynamoDB key
         Sort key value : ${timestamp}
    Write message data to this column : Payload    // it creates a 'Payload' col and entire json
    
- the payload from DynamoDB to lambda is in qmtt_payload.json and in lambda publishDeviceStatus Test qmtt_payload2
- at AWS IoT Test subscribe Topic 'sdk/test/Python' can see the python messages

##### known issues:
- change topic to 'device/staus' not work
- change clientId not work

### curl

In [36]:
pushd /home/ming/all_dev/AWS_IoT/myThings/IoTTestDevice/connect_device_package
curl -v --tlsv1.2 --cacert root-CA.crt \--cert IoTTestDevice.cert.pem \
 --key IoTTestDevice.private.key -X POST \
 -d "{\"myid\": \"curl1002\",\"timestamp\": \"29-10-2019 23:04:20\",\"sequence\": 1014,\"status\": \"Unknown\"}" \
 https://a3a13fa4bs6nw5-ats.iot.us-west-2.amazonaws.com:8443/topics/topic_1?qos=1
 
popd
device/status

~/all_dev/AWS_IoT/myThings/IoTTestDevice/connect_device_package ~/all_dev/AWS_IoT/project4greg ~/all_dev/AWS_IoT/project4greg
Note: Unnecessary use of -X or --request, POST is already inferred.
*   Trying 52.89.125.11...
* TCP_NODELAY set
* Connected to a3a13fa4bs6nw5-ats.iot.us-west-2.amazonaws.com (52.89.125.11) port 8443 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: root-CA.crt
  CApath: /etc/ssl/certs
* TLSv1.2 (OUT), TLS handshake, Client hello (1):
* TLSv1.2 (IN), TLS handshake, Server hello (2):
* TLSv1.2 (IN), TLS handshake, Certificate (11):
* TLSv1.2 (IN), TLS handshake, Server key exchange (12):
* TLSv1.2 (IN), TLS handshake, Request CERT (13):
* TLSv1.2 (IN), TLS handshake, Server finished (14):
* TLSv1.2 (OUT), TLS handshake, Certificate (11):
* TLSv1.2 (OUT), TLS handshake, Client key exchange (16):
* TLSv1.2 (OUT), TLS handshake, CERT verify (15):
* TLSv1.2 (OUT), TLS change cipher, Client hello (1):
* TLS

: 127

In [31]:
pushd /home/ming/all_dev/AWS_IoT/project4greg
./demo.sh
popd

~/all_dev/AWS_IoT/project4greg ~/all_dev/AWS_IoT/project4greg

Running pub/sub sample application...
2019-10-30 12:35:56,788 - AWSIoTPythonSDK.core.protocol.internal.clients - DEBUG - Initializing MQTT layer...
2019-10-30 12:35:56,788 - AWSIoTPythonSDK.core.protocol.internal.clients - DEBUG - Registering internal event callbacks to MQTT layer...
2019-10-30 12:35:56,788 - AWSIoTPythonSDK.core.protocol.mqtt_core - INFO - MqttCore initialized
2019-10-30 12:35:56,788 - AWSIoTPythonSDK.core.protocol.mqtt_core - INFO - Client id: basicPubSub
2019-10-30 12:35:56,788 - AWSIoTPythonSDK.core.protocol.mqtt_core - INFO - Protocol version: MQTTv3.1.1
2019-10-30 12:35:56,788 - AWSIoTPythonSDK.core.protocol.mqtt_core - INFO - Authentication type: TLSv1.2 certificate based Mutual Auth.
2019-10-30 12:35:56,788 - AWSIoTPythonSDK.core.protocol.mqtt_core - INFO - Configuring endpoint...
2019-10-30 12:35:56,788 - AWSIoTPythonSDK.core.protocol.mqtt_core - INFO - Configuring certificates...
2019-10-30 12:35:

2019-10-30 12:48:09,264 - AWSIoTPythonSDK.core.protocol.internal.workers - DEBUG - Produced [suback] event
2019-10-30 12:48:09,264 - AWSIoTPythonSDK.core.protocol.internal.workers - DEBUG - Dispatching [suback] event
2019-10-30 12:48:09,264 - AWSIoTPythonSDK.core.protocol.internal.clients - DEBUG - Invoking custom event callback...
2019-10-30 12:48:09,264 - AWSIoTPythonSDK.core.protocol.internal.clients - DEBUG - This custom event callback is for pub/sub/unsub, removing it after invocation...
2019-10-30 12:48:11,377 - AWSIoTPythonSDK.core.protocol.internal.workers - DEBUG - Produced [disconnect] event
2019-10-30 12:48:11,377 - AWSIoTPythonSDK.core.protocol.internal.workers - DEBUG - Dispatching [disconnect] event
2019-10-30 12:48:11,377 - AWSIoTPythonSDK.core.protocol.connection.cores - DEBUG - backOff: current backoff time is: 4 sec.
2019-10-30 12:48:11,377 - AWSIoTPythonSDK.core.protocol.internal.clients - DEBUG - Invoking custom event callback...
2019-10-30 12:48:15,531 - AWSIoTPyth